# LangChain RAG: From Basics to Production-Ready RAG Chatbot

**Updated for LangChain v1.2.4 (January 2026)**

This notebook is a modernized version of the [FutureSmart.ai RAG Tutorial](https://blog.futuresmart.ai/langchain-rag-from-basics-to-production-ready-rag-chatbot), updated to use the latest LangChain syntax and best practices.

## What's New in This Version
- ✅ Uses `create_agent` from `langchain.agents` (modern agent API)
- ✅ Modern `create_retriever_tool` from `langchain_core.tools`
- ✅ `InMemorySaver` checkpointer for conversation memory
- ✅ GPT-4o vision support for PDFs with diagrams/tables
- ✅ Includes local model alternatives (Ollama)
- ✅ Advanced PDF processing (tables, diagrams, images)

## Tutorial Outline
1. Setup & Installation
2. LangChain Basics (LLM, Prompts, LCEL)
3. Document Processing (including vision-based PDF extraction)
4. Vector Store & Retriever
5. Building a RAG Agent (Modern Approach)
6. Conversational RAG with Memory
7. Production Considerations (Streaming, Multi-tool Agents)

---
## 1. Setup & Installation

Install the required packages. We use the modular LangChain packages for better dependency management.

In [ ]:
# Install required packages (LangChain v1.2.4, January 2026)
# NOTE: Run this cell once, then restart runtime if needed

# Core packages
!pip install -qU 'langchain>=1.2.4' langchain-openai langchain-chroma langgraph
!pip install -qU langchain-community pypdf docx2txt sentence-transformers

# For advanced PDF processing (tables, diagrams, images)
!pip install -qU pymupdf pillow pdfplumber

# For local models using Ollama (optional but recommended)
!pip install -qU langchain-ollama

In [ ]:
# Set up your API keys
import os

# Option 1: Set directly (not recommended for production)
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

# Option 2: Load from .env file (recommended)
# from dotenv import load_dotenv
# load_dotenv()

---
## 2. LangChain Basics

Before diving into RAG, let's understand the core LangChain components.

### 2.1 Working with LLMs

In [ ]:
# Initialize the LLM
from langchain_openai import ChatOpenAI

# Using OpenAI's GPT-4o-mini (cost-effective choice for tutorials)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Simple invocation
response = llm.invoke("What is Retrieval Augmented Generation?")
print(response.content)

In [ ]:
# ALTERNATIVE: Using local models with Ollama
# Ollama provides free, local inference with models like Llama 3.2, DeepSeek, Mistral, etc.

# To use Ollama:
# 1. Install Ollama: https://ollama.com/download
# 2. Pull a model: ollama pull llama3.2
# 3. Uncomment the code below

USE_OLLAMA = False  # Set to True to use local Ollama instead of OpenAI

if USE_OLLAMA:
    from langchain_ollama import ChatOllama
    
    # For text-only tasks, use a standard model
    llm = ChatOllama(
        model="llama3.2",      # or "mistral", "deepseek-r1:8b", "qwen2.5"
        temperature=0
    )
    
    # For vision/multimodal tasks (PDFs with images), use a vision model
    llm_vision = ChatOllama(
        model="llava",         # or "llava:13b", "bakllava"
        temperature=0
    )
    
    response = llm.invoke("What is RAG?")
    print("Ollama response:", response.content)
else:
    print("Using OpenAI (set USE_OLLAMA=True to use local Ollama)")

### 2.2 Prompt Templates

Prompt templates help structure our prompts with dynamic variables.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    "You are an expert on {topic}. Explain {concept} in simple terms."
)

# Format the prompt
formatted_prompt = prompt.format(topic="machine learning", concept="embeddings")
print("Formatted prompt:")
print(formatted_prompt)

### 2.3 Output Parsers

Output parsers help structure the LLM's response.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# StrOutputParser extracts just the text content
output_parser = StrOutputParser()

# Parse the response
parsed_output = output_parser.parse(response.content)
print(type(parsed_output))  # <class 'str'>

### 2.4 LCEL (LangChain Expression Language) Chains

LCEL allows us to chain components together using the pipe (`|`) operator.

In [ ]:
# Create an LCEL chain: prompt -> llm -> parser
chain = prompt | llm | output_parser

# Invoke the chain
result = chain.invoke({"topic": "AI", "concept": "vector databases"})
print(result)

### 2.5 Structured Output (Pydantic)

For more complex use cases, we can get structured JSON output.

In [ ]:
from typing import List
from pydantic import BaseModel, Field

# Define the output structure
class ConceptExplanation(BaseModel):
    concept: str = Field(description="The concept being explained")
    definition: str = Field(description="A concise definition")
    key_points: List[str] = Field(description="Key points to remember")
    difficulty: str = Field(description="Difficulty level: beginner, intermediate, advanced")

# Use structured output
structured_llm = llm.with_structured_output(ConceptExplanation)

# Get structured response
structured_response = structured_llm.invoke(
    "Explain what a Vector Database is and why it's important for RAG"
)
print(f"Concept: {structured_response.concept}")
print(f"Definition: {structured_response.definition}")
print(f"Key Points: {structured_response.key_points}")
print(f"Difficulty: {structured_response.difficulty}")

---
## 3. Document Processing

To build a RAG system, we need to load and process documents.

### 3.1 Loading Documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader

# Example: Load a PDF file
# Replace with your own file path
pdf_path = "sample_document.pdf"  # <-- Update this path

# Uncomment to use with a real PDF:
# loader = PyPDFLoader(pdf_path)
# documents = loader.load()
# print(f"Loaded {len(documents)} pages")

# For this demo, we'll create sample documents
from langchain_core.documents import Document

# Sample documents for demonstration
sample_documents = [
    Document(
        page_content="LangChain is a framework for developing applications powered by language models. It provides tools for prompt management, chains, and agents.",
        metadata={"source": "langchain_intro.txt", "page": 1}
    ),
    Document(
        page_content="RAG (Retrieval Augmented Generation) combines retrieval and generation to produce more accurate and up-to-date responses. It works by retrieving relevant documents from a knowledge base.",
        metadata={"source": "rag_overview.txt", "page": 1}
    ),
    Document(
        page_content="Vector databases store data as high-dimensional vectors, enabling similarity search. Popular options include Chroma, Pinecone, and Weaviate.",
        metadata={"source": "vector_db.txt", "page": 1}
    ),
    Document(
        page_content="Embeddings are numerical representations of text that capture semantic meaning. OpenAI embeddings and sentence-transformers are commonly used.",
        metadata={"source": "embeddings.txt", "page": 1}
    ),
    Document(
        page_content="LangGraph is a library for building stateful, multi-actor applications. It powers LangChain's agent framework with features like persistence and streaming.",
        metadata={"source": "langgraph.txt", "page": 1}
    )
]

print(f"Created {len(sample_documents)} sample documents for demonstration")

### 3.2 Advanced PDF Loading: Tables, Diagrams, and Images

Standard PDF loaders only extract text. For PDFs with **tables, diagrams, flowcharts, or images**, we need specialized approaches:

1. **Unstructured** - Best for complex layouts with tables
2. **PDFPlumber** - Excellent table extraction
3. **Vision Models (GPT-4o)** - Best for diagrams and visual content

In [ ]:
# ============================================================
# ADVANCED PDF PROCESSING: Tables, Diagrams, and Images
# ============================================================
# For production PDFs with complex layouts, install these:
# !pip install -qU unstructured[pdf] pdf2image pdfplumber pymupdf pillow

# Option 1: Unstructured - Best for complex layouts and tables
# ------------------------------------------------------------
"""
from langchain_community.document_loaders import UnstructuredPDFLoader

# hi_res mode uses vision models to understand layout
loader = UnstructuredPDFLoader(
    "complex_document.pdf",
    mode="elements",           # Preserves document structure
    strategy="hi_res",         # Uses OCR + layout detection
    extract_images_in_pdf=True # Extracts embedded images
)
docs = loader.load()

# Elements are categorized: Title, NarrativeText, Table, Image, etc.
for doc in docs:
    print(f"Type: {doc.metadata.get('category')}")
    print(f"Content: {doc.page_content[:100]}...")
"""

# Option 2: PDFPlumber - Excellent for table extraction
# ------------------------------------------------------------
"""
import pdfplumber
from langchain_core.documents import Document

def extract_tables_from_pdf(pdf_path):
    tables_as_docs = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            for j, table in enumerate(tables):
                # Convert table to markdown format
                if table:
                    headers = table[0]
                    rows = table[1:]
                    md_table = "| " + " | ".join(str(h) for h in headers) + " |\\n"
                    md_table += "| " + " | ".join("---" for _ in headers) + " |\\n"
                    for row in rows:
                        md_table += "| " + " | ".join(str(cell) for cell in row) + " |\\n"
                    
                    tables_as_docs.append(Document(
                        page_content=md_table,
                        metadata={"source": pdf_path, "page": i+1, "table": j+1, "type": "table"}
                    ))
    return tables_as_docs

# Usage:
# table_docs = extract_tables_from_pdf("report_with_tables.pdf")
"""

print("Advanced PDF loading methods defined (uncomment to use)")

### 3.3 Vision-Based PDF Processing with GPT-4o

For PDFs with **diagrams, flowcharts, architecture diagrams, or infographics**, the best approach is to:
1. Convert PDF pages to images
2. Use **GPT-4o** (vision model) to understand and describe the visual content
3. Store the descriptions for RAG retrieval

**GPT-4o** is the recommended model because:
- Native multimodal support (text + images)
- Excellent at understanding diagrams, charts, and tables
- Can extract structured information from visual content

In [ ]:
# ============================================================
# GPT-4o VISION: Process PDFs with Diagrams and Visual Content
# ============================================================
# Required: pip install pdf2image pymupdf pillow

import base64
from io import BytesIO
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.documents import Document

# Use GPT-4o for vision tasks (best for diagrams/tables)
# GPT-4o has native multimodal support - handles text AND images
vision_llm = ChatOpenAI(
    model="gpt-4o",  # Full GPT-4o for best visual understanding
    temperature=0,
    max_tokens=4096
)

def process_pdf_with_vision(pdf_path: str) -> list[Document]:
    """
    Process a PDF by converting pages to images and using GPT-4o
    to extract text, describe diagrams, and understand visual content.
    """
    try:
        import fitz  # PyMuPDF
        from PIL import Image
    except ImportError:
        print("Install required packages: pip install pymupdf pillow")
        return []
    
    documents = []
    pdf = fitz.open(pdf_path)
    
    for page_num in range(len(pdf)):
        page = pdf[page_num]
        
        # Convert page to image (high resolution for better OCR)
        pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # 2x zoom
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # Convert to base64 for GPT-4o
        buffer = BytesIO()
        img.save(buffer, format="PNG")
        img_base64 = base64.b64encode(buffer.getvalue()).decode("utf-8")
        
        # Use GPT-4o to analyze the page
        message = HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": """Analyze this PDF page and extract ALL content:

1. **Text Content**: Extract all readable text, preserving structure
2. **Tables**: If there are tables, convert them to markdown format
3. **Diagrams/Flowcharts**: Describe any diagrams, flowcharts, or visual elements in detail
4. **Images**: Describe any images and their relevance to the content
5. **Key Information**: Highlight important data, figures, or conclusions

Format your response clearly with sections for each type of content found."""
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{img_base64}",
                        "detail": "high"  # High detail for better accuracy
                    }
                }
            ]
        )
        
        response = vision_llm.invoke([message])
        
        documents.append(Document(
            page_content=response.content,
            metadata={
                "source": pdf_path,
                "page": page_num + 1,
                "processing": "gpt-4o-vision",
                "has_visual_content": True
            }
        ))
        
        print(f"Processed page {page_num + 1}/{len(pdf)}")
    
    pdf.close()
    return documents

# Example usage (uncomment with a real PDF):
# vision_docs = process_pdf_with_vision("diagram_heavy_document.pdf")
# print(f"Processed {len(vision_docs)} pages with vision analysis")

print("GPT-4o vision processing function defined!")
print("Use: vision_docs = process_pdf_with_vision('your_pdf.pdf')")

### 3.2 Splitting Documents

Large documents need to be split into smaller chunks for effective retrieval.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configure the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Maximum characters per chunk
    chunk_overlap=100,     # Overlap between chunks for context
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Split priority
)

# Split documents (for demo, our docs are already small)
splits = text_splitter.split_documents(sample_documents)
print(f"Split into {len(splits)} chunks")

# Preview a chunk
print(f"\nSample chunk content:\n{splits[0].page_content}")

---
## 4. Vector Store & Retriever

### 4.1 Creating Embeddings

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Initialize embeddings (uses OpenAI's text-embedding-3-small by default in 2026)
embeddings = OpenAIEmbeddings()

# ALTERNATIVE: Free local embeddings using sentence-transformers
# from langchain_community.embeddings import SentenceTransformerEmbeddings
# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Test embedding
test_embedding = embeddings.embed_query("What is RAG?")
print(f"Embedding dimension: {len(test_embedding)}")

### 4.2 Setting Up the Vector Store

In [ ]:
# MODERN: Use langchain_chroma (dedicated package)
from langchain_chroma import Chroma

# Create vector store from documents
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db"  # Persists to disk
)

print("Vector store created and persisted!")

### 4.3 Creating a Retriever

In [ ]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",  # or "mmr" for diversity
    search_kwargs={"k": 3}     # Return top 3 results
)

# Test retrieval
query = "What is LangChain used for?"
retrieved_docs = retriever.invoke(query)

print(f"Query: {query}\n")
print("Retrieved documents:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n{i}. [{doc.metadata.get('source', 'unknown')}]")
    print(f"   {doc.page_content[:150]}...")

---
## 5. Building a RAG Agent (Modern Approach)

### Why Use Agents Instead of Chains?

The original tutorial used `create_retrieval_chain`. While still valid, the modern approach uses **agents** because:

1. **Flexibility**: Agent decides WHEN to retrieve (not every query needs retrieval)
2. **Extensibility**: Easy to add more tools (web search, calculator, etc.)
3. **Production-ready**: Built on LangGraph with persistence, streaming, etc.
4. **Middleware**: v1.1+ supports pre/post model hooks for guardrails

### 5.1 Creating a Retriever Tool

In [ ]:
# LangChain v1.2.4: create_retriever_tool is now in langchain_core.tools
from langchain_core.tools import create_retriever_tool

# Wrap the retriever as a tool the agent can use
retriever_tool = create_retriever_tool(
    retriever,
    name="knowledge_base_search",
    description="""Search the knowledge base for information about LangChain, 
    RAG, vector databases, and embeddings. Use this tool when you need to 
    find specific information from the documents."""
)

print(f"Tool created: {retriever_tool.name}")
print(f"Description: {retriever_tool.description}")

### 5.2 Creating the RAG Agent

Now we use the modern `create_agent` API from `langchain.agents` (LangChain v1.2.4, January 2026).

In [ ]:
# LangChain v1.2.4 (2026): Using create_agent from langgraph.prebuilt
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# Define the system prompt for the RAG agent
RAG_SYSTEM_PROMPT = """You are a helpful AI assistant with access to a knowledge base.

When answering questions:
1. ALWAYS use the knowledge_base_search tool to find relevant information
2. Base your answers on the retrieved documents
3. If the information isn't in the knowledge base, say so honestly
4. Cite the source when providing information

Be concise but thorough in your responses.
"""

# Create the model instance
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create the agent
# NOTE: create_agent comes from langgraph.prebuilt
# The system_prompt parameter adds the system prompt
rag_agent = create_agent(
    model=model,                         # LangChain model instance
    tools=[retriever_tool],              # List of tools the agent can use
    system_prompt=RAG_SYSTEM_PROMPT     # System prompt as string
)

print("RAG Agent created successfully!")

### 5.3 Using the RAG Agent

In [ ]:
# Invoke the agent with a question
from langchain_core.messages import HumanMessage

question = "What is RAG and how does it work?"

response = rag_agent.invoke({
    "messages": [HumanMessage(content=question)]
})

# Extract the final answer
# The response contains the full message history
final_message = response["messages"][-1]
print(f"Question: {question}")
print(f"\nAnswer:\n{final_message.content}")

In [ ]:
# Another example query
question2 = "What are the popular vector databases I can use?"

response2 = rag_agent.invoke({
    "messages": [HumanMessage(content=question2)]
})

print(f"Question: {question2}")
print(f"\nAnswer:\n{response2['messages'][-1].content}")

---
## 6. Conversational RAG with Memory

For a true chatbot experience, we need conversation memory.

### 6.1 Adding Memory with Checkpointing

In [ ]:
# CORRECT: InMemorySaver from langgraph.checkpoint.memory
from langgraph.checkpoint.memory import InMemorySaver

# Create a memory saver for persistence
checkpointer = InMemorySaver()

# Create agent with memory (LangChain v1.2.4)
# NOTE: create_agent comes from langgraph.prebuilt
conversational_agent = create_agent(
    model=model,                          # Reuse the model from above
    tools=[retriever_tool],
    system_prompt=RAG_SYSTEM_PROMPT,
    checkpointer=checkpointer,            # Enable conversation persistence
)

print("Conversational agent with memory created!")

### 6.2 Multi-Turn Conversation

In [ ]:
# Define a session/thread ID for conversation tracking
config = {"configurable": {"thread_id": "user-session-123"}}

# First message
print("=" * 50)
print("Turn 1")
print("=" * 50)

response1 = conversational_agent.invoke(
    {"messages": [HumanMessage(content="What is LangChain?")]},
    config=config
)
print(f"User: What is LangChain?")
print(f"Assistant: {response1['messages'][-1].content}")

In [ ]:
# Follow-up question (agent remembers previous context)
print("\n" + "=" * 50)
print("Turn 2 (Follow-up)")
print("=" * 50)

response2 = conversational_agent.invoke(
    {"messages": [HumanMessage(content="How does it relate to LangGraph?")]},
    config=config  # Same thread_id maintains context
)
print(f"User: How does it relate to LangGraph?")
print(f"Assistant: {response2['messages'][-1].content}")

In [ ]:
# Third turn
print("\n" + "=" * 50)
print("Turn 3 (Another follow-up)")
print("=" * 50)

response3 = conversational_agent.invoke(
    {"messages": [HumanMessage(content="What about vector databases?")]},
    config=config
)
print(f"User: What about vector databases?")
print(f"Assistant: {response3['messages'][-1].content}")

### 6.3 Multi-User Support

Different users get different conversation contexts by using unique thread IDs.

In [ ]:
# User A conversation
user_a_config = {"configurable": {"thread_id": "user-alice-001"}}

response_a = conversational_agent.invoke(
    {"messages": [HumanMessage(content="Tell me about embeddings")]},
    config=user_a_config
)
print(f"User Alice: Tell me about embeddings")
print(f"Assistant: {response_a['messages'][-1].content}")

In [ ]:
# User B has a completely separate conversation
user_b_config = {"configurable": {"thread_id": "user-bob-002"}}

response_b = conversational_agent.invoke(
    {"messages": [HumanMessage(content="What is RAG?")]},
    config=user_b_config
)
print(f"\nUser Bob: What is RAG?")
print(f"Assistant: {response_b['messages'][-1].content}")

### 6.4 Query Condensing for Follow-up Questions

> **The Problem**: When a user asks *"How does it work?"* after discussing LangChain, the retriever only sees the literal text "How does it work?" — without any context about what "it" refers to. This leads to poor or irrelevant retrieval results.

**Solution**: Use a **query condenser** (also called a *history-aware retriever*) that automatically rewrites follow-up questions into standalone queries by incorporating conversation history.

| Turn | User Question | Condensed Query |
|------|---------------|-----------------|
| 1 | "What is LangChain?" | "What is LangChain?" |
| 2 | "How does it work?" | "How does LangChain work?" |
| 3 | "What about with vector DBs?" | "How does LangChain work with vector databases?" |

In [ ]:
# Step 1: Define the Condensing Prompt Template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

CONDENSE_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """Given a chat history and the latest user question, 
rewrite the question to be a standalone query that captures the full context.

Rules:
- If the question references previous context (e.g., "it", "that", "this"), expand it
- If the question is already standalone, return it unchanged
- Only output the rewritten question, nothing else"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# Create the query condensing chain
query_condenser = CONDENSE_PROMPT | llm | StrOutputParser()

print("✅ Query condenser chain created")

In [ ]:
# Step 2: Create History-Aware Retriever
from langchain.chains import create_history_aware_retriever

# This wraps the retriever to first condense the query using chat history
history_aware_retriever = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,
    prompt=CONDENSE_PROMPT
)

print("✅ History-aware retriever created")

In [ ]:
# Step 3: Build Agent with History-Aware Retrieval
from langchain.tools.retriever import create_retriever_tool

# Create a tool that uses the history-aware retriever
condensing_retriever_tool = create_retriever_tool(
    retriever=history_aware_retriever,
    name="search_documents_with_context",
    description="Search documents using a context-aware query. Use this for follow-up questions."
)

# Build the agent with the new tool
agent_with_condensing = create_react_agent(
    model=llm,
    tools=[condensing_retriever_tool],
    prompt="You are a helpful assistant. Use the search tool to answer questions."
)

print("✅ Agent with query condensing ready")

In [ ]:
# Step 4: Test Multi-Turn Conversation
from langchain_core.messages import HumanMessage, AIMessage

# Simulate a 3-turn conversation demonstrating query condensing
chat_history = []
test_questions = [
    "What is LangChain?",
    "How does it work?",           # "it" → LangChain
    "What about vector databases?" # continues context
]

print("=" * 60)
print("🔄 MULTI-TURN CONVERSATION TEST")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n👤 Turn {i}: {question}")
    
    # Invoke agent with chat history
    response = agent_with_condensing.invoke({
        "messages": chat_history + [HumanMessage(content=question)]
    })
    
    # Extract and display response
    answer = response["messages"][-1].content
    print(f"🤖 Assistant: {answer[:200]}..." if len(answer) > 200 else f"🤖 Assistant: {answer}")
    
    # Update chat history
    chat_history.extend([
        HumanMessage(content=question),
        AIMessage(content=answer)
    ])

print("\n" + "=" * 60)
print("✅ Query condensing working - follow-up questions understood!")
print("=" * 60)

---
## 7. Production Considerations

### 7.1 Streaming Responses

For better UX, stream responses token-by-token.

In [ ]:
# Streaming example (correct v1.2.4 pattern)
print("Streaming response:")
print("-" * 50)

# Use stream_mode="values" for cleaner message access
for step in conversational_agent.stream(
    {"messages": [HumanMessage(content="Give me a summary of what I've learned")]},
    config={"configurable": {"thread_id": "user-session-123"}},
    stream_mode="values",
):
    # Access the last message in the step
    last_message = step["messages"][-1]
    if hasattr(last_message, "content") and last_message.content:
        # For final AI response, print it
        if isinstance(last_message, AIMessage):
            print(last_message.content)

print("-" * 50)

### 7.2 Middleware (NEW in LangChain v1.1+)

Middleware allows you to add pre/post processing hooks to your agent.

In [ ]:
# Advanced: Pre-processing with custom node wrapper
# LangGraph uses custom nodes for middleware-like behavior

from langchain_core.messages import HumanMessage, AIMessage

def with_logging(agent):
    """Wrapper that adds logging around agent invocations."""
    original_invoke = agent.invoke
    
    def logged_invoke(inputs, config=None, **kwargs):
        print(f"[LOG] Processing {len(inputs.get('messages', []))} messages...")
        result = original_invoke(inputs, config, **kwargs)
        print(f"[LOG] Response received with {len(result.get('messages', []))} messages.")
        return result
    
    agent.invoke = logged_invoke
    return agent

# Create agent with logging wrapper
agent_with_logging = with_logging(create_agent(
    model=model,
    tools=[retriever_tool],
    system_prompt=RAG_SYSTEM_PROMPT,
))

# Test it
print("Testing agent with logging:")
result = agent_with_logging.invoke({
    "messages": [HumanMessage(content="What is LangChain?")]
})
print(f"Answer: {result['messages'][-1].content[:200]}...")

### 7.3 Adding More Tools

One advantage of the agent approach is easy tool extension.

In [ ]:
from langchain_core.tools import tool

# Custom tool example: Get current date
@tool
def get_current_date() -> str:
    """Get the current date. Use when asked about today's date."""
    from datetime import date
    return date.today().strftime("%B %d, %Y")

# Create agent with multiple tools
multi_tool_agent = create_agent(
    model=model,
    tools=[retriever_tool, get_current_date],
    system_prompt="""You are a helpful assistant with access to:
    1. A knowledge base about LangChain and RAG (knowledge_base_search)
    2. A tool to get the current date (get_current_date)
    
    Use the appropriate tool based on the user's question."""
)

# Test with a date question
response = multi_tool_agent.invoke({
    "messages": [HumanMessage(content="What's today's date?")]
})
print(f"Response: {response['messages'][-1].content}")

### 7.4 Error Handling

In [ ]:
def safe_query(agent, question: str, config: dict = None) -> str:
    """Safely query the agent with error handling."""
    try:
        response = agent.invoke(
            {"messages": [HumanMessage(content=question)]},
            config=config or {}
        )
        return response["messages"][-1].content
    except Exception as e:
        return f"Error processing query: {str(e)}"

# Usage
result = safe_query(rag_agent, "What frameworks are mentioned in the docs?")
print(result)

---
## Summary

In this tutorial (updated for **LangChain v1.2.4, January 2026**), we covered:

1. **LangChain Basics**: LLMs, prompts, parsers, and LCEL chains
2. **Document Processing**: Loading and splitting documents (including vision-based PDF extraction)
3. **Vector Stores**: Creating embeddings and setting up Chroma
4. **RAG Agents**: Using the modern `create_react_agent` + `create_retriever_tool` pattern
5. **Conversation Memory**: Multi-turn conversations with `InMemorySaver`
6. **Query Condensing**: History-aware retriever for better follow-up question handling
7. **Production Features**: Streaming, custom wrappers, multi-tool agents

### Key Takeaways (v1.2.4 Updates)

| Old Pattern (Pre-v1) | Modern Pattern (v1.2.4) |
|------------------------|----------------------------|
| `create_retrieval_chain` | `create_react_agent` from `langgraph.prebuilt` + retriever tool |
| Manual history management | `InMemorySaver` from `langgraph.checkpoint.memory` |
| `langchain_community.vectorstores` | `langchain_chroma` |
| No state_modifier | `state_modifier=system_prompt` for system instructions |
| Complex memory setup | Simple `checkpointer=InMemorySaver()` |
| No query condensing | `create_history_aware_retriever` for follow-ups |

### Key Imports Summary (v1.2.4)

```python
# Agent creation
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import create_retriever_tool  # v1.2.4: moved to langchain_core
from langchain_core.tools import tool  # For custom tools

# Query Condensing (for follow-up questions)
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Models - Choose ONE provider:
# OpenAI:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # text
vision_llm = ChatOpenAI(model="gpt-4o", temperature=0)  # vision

# OR Ollama (local):
from langchain_ollama import ChatOllama, OllamaEmbeddings
llm = ChatOllama(model="llama3.2")  # text: llama3.2, mistral, deepseek-r1
llm_vision = ChatOllama(model="llava")  # vision: llava, bakllava

# Memory/Checkpointing
from langgraph.checkpoint.memory import InMemorySaver

# Messages
from langchain_core.messages import HumanMessage, AIMessage

# Vector stores & Embeddings
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings  # or OllamaEmbeddings
```

### Next Steps
- Add more document loaders (web pages, databases)
- Implement user authentication for thread management
- Add observability with LangSmith
- Deploy as an API with FastAPI